In [3]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import shutil
import torch
from ultralytics import YOLO


In [4]:

# Define paths
image_dir = "/home/murtaza/University_Data/deep_learning/assignment3/Task_02_vehicles_dataset/images"
label_dir = "/home/murtaza/University_Data/deep_learning/assignment3/Task_02_vehicles_dataset/labels"

# Get all image files
image_files = sorted(os.listdir(image_dir))
len(image_files)
label_files = sorted(os.listdir(label_dir))
len(label_files)

923

In [5]:
# Optional sanity check
for img_file, lbl_file in zip(image_files, label_files):
    img_name = os.path.splitext(img_file)[0]
    lbl_name = os.path.splitext(lbl_file)[0]
    if img_name != lbl_name:
        print(f"Mismatch: {img_name} vs {lbl_name}")


In [6]:
data = []
for img_file, lbl_file in zip(image_files, label_files):
    img_path = os.path.join(image_dir, img_file)
    lbl_path = os.path.join(label_dir, lbl_file)
    data.append([img_path, lbl_path])
df = pd.DataFrame(data, columns=["image_path", "label_path"])
df.to_csv("TrafficDetectionData.csv", index=False)


In [7]:

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)


In [14]:

# Get all file extensions
file_extensions = set([os.path.splitext(f)[1] for f in os.listdir(image_dir)])
print("Unique file extensions found:", file_extensions)


Unique file extensions found: {'.png', '.jpg'}


In [ ]:


# Create YOLO-style folder structure
base_dir = "yolo_dataset"
for split in ["train", "val"]:
    os.makedirs(os.path.join(base_dir, "images", split), exist_ok=True)
    os.makedirs(os.path.join(base_dir, "labels", split), exist_ok=True)

# Function to copy files to YOLO folder
def copy_files(df, split):
    for _, row in df.iterrows():
        img_dest = os.path.join(base_dir, "images", split, os.path.basename(row["image_path"]))
        lbl_dest = os.path.join(base_dir, "labels", split, os.path.basename(row["label_path"]))
        shutil.copy(row["image_path"], img_dest)
        shutil.copy(row["label_path"], lbl_dest)

# Copy training and validation data
copy_files(train_df, "train")
copy_files(val_df, "val")


In [16]:
data_yaml = """
path: /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset
train: images/train
val: images/val

nc: 1
names: ['vehicle']
"""

with open("/home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/data.yaml", "w") as f:
    f.write(data_yaml.strip())

print("data.yaml created.")


data.yaml created.


In [ ]:
torch.cuda.empty_cache()


In [ ]:

# Train YOLOv8-nano
model_nano = YOLO('yolov8n.pt')
model_nano.train(data='yolo_dataset/data.yaml', epochs=50, imgsz=640,name='yolov8n_train')

# Train YOLOv8-small
model_small = YOLO('yolov8s.pt')
model_small.train(data='yolo_dataset/data.yaml', epochs=50, imgsz=640,name='yolov8s_train')


Ultralytics 8.3.124 🚀 Python-3.12.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5815MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=yolo_dataset/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolov8n_train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, 

train: Scanning /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/labels/train... 738 images, 5 backgrounds, 705 corrupt: 100%|██████████| 738/738 [00:00<00:00, 4849.29it/s]

train: /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/images/train/00 (10).jpg: ignoring corrupt image/label: Label class 5 exceeds dataset class count 1. Possible class labels are 0-0
train: /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/images/train/00 (100).jpg: ignoring corrupt image/label: Label class 6 exceeds dataset class count 1. Possible class labels are 0-0
train: /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/images/train/00 (102).jpg: ignoring corrupt image/label: Label class 6 exceeds dataset class count 1. Possible class labels are 0-0
train: /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/images/train/00 (103).jpg: ignoring corrupt image/label: Label class 6 exceeds dataset class count 1. Possible class labels are 0-0
train: /home/murtaza/University_Data/deep_learning/assignment

val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3272.5±1618.0 MB/s, size: 395.1 KB)


val: Scanning /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/labels/val... 185 images, 2 backgrounds, 173 corrupt: 100%|██████████| 185/185 [00:00<00:00, 3302.87it/s]

...

                   all         12         43      0.924       0.86      0.935      0.757



50 epochs completed in 0.016 hours.
Optimizer stripped from runs/detect/yolov8s_train/weights/last.pt, 22.5MB
Optimizer stripped from runs/detect/yolov8s_train/weights/best.pt, 22.5MB

Validating runs/detect/yolov8s_train/weights/best.pt...
Ultralytics 8.3.124 🚀 Python-3.12.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5815MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  9.25it/s]


                   all         12         43      0.836      0.953      0.937       0.79
Speed: 0.2ms preprocess, 3.0ms inference, 0.0ms loss, 1.7ms postprocess per image
Results saved to runs/detect/yolov8s_train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x747db3319730>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

TESTING

In [9]:

# Load the best models from training
model_nano = YOLO('runs/detect/yolov8n_train/weights/best.pt')  # Adjust the path to your run
model_small = YOLO('runs/detect/yolov8s_train/weights/best.pt')  # Adjust the path to your run

# You can print a quick summary of both models
print(f"Nano Model: {model_nano}")
print(f"Small Model: {model_small}")

Nano Model: YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, tra

In [20]:
# Evaluate models
metrics_nano = model_nano.val(name = 'yolov8n_val')
metrics_small = model_small.val(name = 'yolov8s_val')

# Print metrics for both models
print("Metrics for Nano Model:", metrics_nano)
print("Metrics for Small Model:", metrics_small)


Ultralytics 8.3.124 🚀 Python-3.12.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5815MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 3990.0±1495.2 MB/s, size: 140.9 KB)


val: Scanning /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/labels/val.cache... 185 images, 2 backgrounds, 173 corrupt: 100%|██████████| 185/185 [00:00<?, ?it/s]

val: /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/images/val/00 (106).jpg: ignoring corrupt image/label: Label class 6 exceeds dataset class count 1. Possible class labels are 0-0
val: /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/images/val/00 (123).jpg: ignoring corrupt image/label: Label class 3 exceeds dataset class count 1. Possible class labels are 0-0
val: /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/images/val/00 (13).jpg: ignoring corrupt image/label: Label class 6 exceeds dataset class count 1. Possible class labels are 0-0
val: /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/images/val/00 (130).jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
val: /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds2404


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.40it/s]


                   all         12         43      0.983      0.837      0.976      0.806
Speed: 0.2ms preprocess, 9.8ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to runs/detect/yolov8n_val
Ultralytics 8.3.124 🚀 Python-3.12.9 torch-2.6.0+cu126 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 5815MiB)
Model summary (fused): 72 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2513.9±469.1 MB/s, size: 116.4 KB)


val: Scanning /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/labels/val.cache... 185 images, 2 backgrounds, 173 corrupt: 100%|██████████| 185/185 [00:00<?, ?it/s]

val: /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/images/val/00 (106).jpg: ignoring corrupt image/label: Label class 6 exceeds dataset class count 1. Possible class labels are 0-0
val: /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/images/val/00 (123).jpg: ignoring corrupt image/label: Label class 3 exceeds dataset class count 1. Possible class labels are 0-0
val: /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/images/val/00 (13).jpg: ignoring corrupt image/label: Label class 6 exceeds dataset class count 1. Possible class labels are 0-0
val: /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/images/val/00 (130).jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
val: /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds2404


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  3.17it/s]


                   all         12         43      0.836      0.953      0.936      0.792
Speed: 0.2ms preprocess, 7.5ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/yolov8s_val
Metrics for Nano Model: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x747d54079730>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.

In [26]:
# For nano model
results_nano = model_nano.predict(
    source='yolo_dataset/images/test_images',
    stream=True,
    save=True,
    conf=0.3,
    name='yolov8n_test'
)

for r in results_nano:
    pass  # This triggers inference and saving

# For small model
results_small = model_small.predict(
    source='yolo_dataset/images/test_images',
    stream=True,
    save=True,
    conf=0.3,
    name='yolov8s_test'
)

for r in results_small:
    pass  # This triggers inference and saving

# This will save the results with bounding boxes in separate folders like:
# runs/detect/predict_nano/ and runs/detect/predict_small/



image 1/278 /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/images/test_images/00 (1).png: 640x608 1 vehicle, 23.0ms
image 2/278 /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/images/test_images/00 (10).png: 640x640 (no detections), 7.2ms
image 3/278 /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/images/test_images/00 (100).png: 384x640 3 vehicles, 6.8ms
image 4/278 /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/images/test_images/00 (11).png: 640x640 1 vehicle, 7.1ms
image 5/278 /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/images/test_images/00 (12).png: 640x640 (no detections), 9.8ms
image 6/278 /home/murtaza/University_Data/deep_learning/assignment3/murtaza_msds24040_03/Task2/yolo_dataset/images/test_images/00 (13).png: 640x640 1 vehicle, 1

In [8]:
image_path = '/home/murtaza/University_Data/deep_learning/assignment3/Task_02_vehicles_dataset/images/00 (103).jpg'

label_path = '/home/murtaza/University_Data/deep_learning/assignment3/Task_02_vehicles_dataset/labels/00 (103).txt'

# visualize_prediction_vs_groundtruth(image_path, label_path)

In [10]:
model_nano.predict(
    source=image_path,
    save=True,
    conf=0.3,
    name='eval'
)


image 1/1 /home/murtaza/University_Data/deep_learning/assignment3/Task_02_vehicles_dataset/images/00 (103).jpg: 384x640 6 vehicles, 34.3ms
Speed: 3.5ms preprocess, 34.3ms inference, 97.3ms postprocess per image at shape (1, 3, 384, 640)
Results saved to runs/detect/eval2


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'vehicle'}
 obb: None
 orig_img: array([[[251, 249, 248],
         [251, 249, 248],
         [251, 249, 248],
         ...,
         [202, 226, 196],
         [202, 226, 196],
         [202, 226, 196]],
 
        [[251, 249, 248],
         [251, 249, 248],
         [251, 249, 248],
         ...,
         [202, 226, 196],
         [202, 226, 196],
         [202, 226, 196]],
 
        [[251, 249, 248],
         [251, 249, 248],
         [251, 249, 248],
         ...,
         [202, 226, 196],
         [202, 226, 196],
         [202, 226, 196]],
 
        ...,
 
        [[128, 135, 150],
         [128, 135, 150],
         [128, 135, 150],
         ...,
         [119, 134, 150],
         [119, 134, 150],
         [119, 134, 150]],
 
        [[128, 135, 150],
         [128, 135, 150],
         [128, 135, 150],
         ...,
         [106, 12